In [1]:
import re
from bs4 import BeautifulSoup
import tensorflow as tf
import jieba
reviews_len_max = 261
import pandas as pd 

In [2]:
with open('token.txt','r') as f:
    token_str = f.read()
token = tf.keras.preprocessing.text.tokenizer_from_json(token_str)

In [3]:
model=tf.keras.models.Sequential([
    #嵌入层：将已经数字化的影评转化为向量
    #向量化目的：实现将词语嵌入多维矩阵，使得语义相近的词语，在空间距离上更接近
    #设置输出的词向量维度为64，输入维度5000，每条文本长度与最长文本相同
    tf.keras.layers.Embedding(output_dim=64,input_dim=5000,input_length=reviews_len_max),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), #RNN的双向封装器，用于对序列进行前向和后向计算
    tf.keras.layers.Dense(64, activation='relu'), #隐藏层，64个神经元，激活函数为relu函数
    #Dropout层以指定的丢弃概率随机丢弃上一层神经元
    tf.keras.layers.Dropout(0.2), #防止过拟合：随机选择丢弃20%神经元
    tf.keras.layers.Dense(1,activation='sigmoid') #输出层
])
#lr指学习率，epsilon指定一个较小的数代替0，防止实现过程除以0
#decay为学习率在每轮训练的衰减因子，取值范围[0,1]，0代表学习率在训练过程中保持不变
#经衰减后的学习率的计算公式为lr_i = lr_start * 1.0 / (1.0 + decay*i) , i为迭代周期，lr_start是lr的初始值

adam = tf.keras.optimizers.Adam(lr=0.0001,epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, #指定优化器——adam
              loss='binary_crossentropy', #指定损失函数——对数损失函数(针对二分类问题)
              metrics=['accuracy']) #设置模型检验的方法——准确度
#恢复训练的数据
model.load_weights('./imdb-classify-lstm/finalmodel_weibo')

In [4]:
def review_to_wordlist(review):
    '''对每条review中的不合法字符进行替换，并进行分词转化成词表'''
    review_text = BeautifulSoup(review, "html.parser").get_text()
    review_text = re.sub('@[\u4e00-\u9fa5]*|#(.)*#'," ", review_text) #先去除用户名和话题标签
    review_text = re.sub('[^\u4e00-\u9fa5]','',review_text) #再去除标点和英文
    review_text = jieba.lcut(review_text) #分词
    return review_text

In [5]:
def display_test_sentiment(text):
    #显示要预测的文本内容
    #print(text)
    #数据规范化
    newtext=review_to_wordlist(text)
    print(' '.join(newtext))
    #将文本转换为数字序列
    input_seq=token.texts_to_sequences([newtext])
    #将序列填充或截断为固定长度的序列
    pad_input_seq=tf.keras.preprocessing.sequence.pad_sequences(input_seq,
                                                   padding='post',
                                                   truncating='post',
                                                   maxlen=reviews_len_max)
    #使用训练好的模型进行预测
    pred=model.predict(pad_input_seq)
    print(pred[0][0])
    #打印预测值
    if pred[0][0]>0.5:
        preValue='positive'
    else:
        preValue='negtive'
    print('predict value:',preValue)

In [6]:
review_text='''但是，最近我们发现有同学存在舞弊行为，
有同学抄袭了别人的报告，有同学带着别人的报告在实验室
做实验。对于这些行为，基础物理实验教学团队经讨论决定
给予相关同学该实验0分的处理。希望我们所有同学以此为
戒，从自身做起，恪守学术道德。
'''
display_test_sentiment(review_text)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.866 seconds.
Prefix dict has been built successfully.
但是 最近 我们 发现 有 同学 存在 舞弊 行为 有 同学 抄袭 了 别人 的 报告 有 同学 带 着 别人 的 报告 在 实验室 做 实验 对于 这些 行为 基础 物理 实验教学 团队 经 讨论 决定 给予 相关 同学 该 实验 分 的 处理 希望 我们 所有 同学 以此为戒 从 自身 做起 恪守 学术 道德
0.14801186
predict value: negtive


In [7]:
review_text='''刚收到北京体育大学前副校长、学术带头人胡杨教授去世的消息。他四五年前就查出胰腺癌，还带病工作。几年前开会坐在他旁边，见他一边开会，一边还在用笔记本电脑给博士生改英文文章。<br><br>学术精英们真的是太累了。<br><br>最近年终统计科研工作量，我们学院发了个论文统计表。表中的81位老师，共发表SCI论文297篇和EI文章73篇（只算第一作者和通讯作者）。最多的一年中就发了17篇。大家想想，我们每个老师的工作量有多大，竞争压力有多大！<br><br>现在还有人说，大学教师有寒暑假，生活很轻松，真的是太不理解这里的辛苦了......<a href="javascript:void(0);" ignore="ignore" suda-data="key=original_blog_unfold&amp;value=click_pickup:4579973128266928:6671336585" class="WB_text_opt" action-type="fl_fold">收起全文<i class="W_ficon ficon_arrow_up">d</i></a>'''
display_test_sentiment(review_text)

刚 收到 北京体育大学 前 副校长 学术 带头人 胡杨 教授 去世 的 消息 他 四五年 前 就 查出 胰腺癌 还 带病 工作 几年 前 开会 坐在 他 旁边 见 他 一边 开会 一边 还 在 用 笔记本电脑 给 博士生 改 英文 文章 学术 精英 们 真的 是 太累 了 最近 年终 统计 科研 工作量 我们 学院 发了 个 论文 统计表 表中 的 位 老师 共 发表 论文 篇 和 文章 篇 只算 第一 作者 和 通讯 作者 最多 的 一年 中 就 发 了 篇 大家 想想 我们 每个 老师 的 工作量 有 多 大 竞争 压力 有 多 大 现在 还有 人 说 大学 教师 有 寒暑假 生活 很 轻松 真的 是 太 不 理解 这里 的 辛苦 了 收起 全文
0.35257977
predict value: negtive


In [8]:
txt = pd.read_csv(r'F:\python\File\DataAnalysis\心理状态分析\微博爬虫\Weibo2_1h.csv',encoding='utf-8')
contents = txt['content']
#n=len(contents)
n=10
for i in range(8,n+8):
    print("第%d条，共%d条" % (i-7,n))
    display_test_sentiment(contents[i*50])
    print()

第1条，共10条
欧冠 超话 今夜 欧冠 小组赛 第轮 对阵 一览 拉齐奥 布鲁日 泽 尼特 多特蒙德 巴萨 尤文 切尔西 克拉 斯诺达尔 基辅 迪纳摩 费伦茨 瓦 罗斯 巴黎 伊斯坦布尔 莱比锡 曼联 雷恩 塞维利亚 收起 全文
0.089334995
predict value: negtive

第2条，共10条
一名 好 的 销售 必须 要 有 极 高 的 瞎聊 的 能力 销售 的 本质 其实 就是 获得 信任 你 一句 话 都 不 跟 别人 说 很 难 获得 信任 如果 要 聊聊 些 什么 客户 的 喜好 兴趣 点 差异 很大 你 最好 什么 都 能 聊 上去 能 说 个 来 因此 平常 看看 闲书 了解 一下 你 不 熟悉 的 历史 体育 地理 人文 等 在 关键 时候 都 有着 意想不到 的 作用 说到底 人生 来 寂寞 能 说些 互相 都 感兴趣 的 就是 信任 的 开始 收起 全文
0.98020977
predict value: positive

第3条，共10条
来 你 不要脸 我们 就 摊开 来 讨论 第一 言而无信 的 是 你 吧 说好 的 热 转前 三 最后 找前 六 企业 诚信 就 那么 一文不值 我 不 觉得 连 自己 说 的话 都 不 遵守 的 企业 有 什么 信用 可言 第二 改变 规则 不 公开 声明 信息 不 对称 的 亏 就 我们 一家 吃 了 什么 意思 啊 米子 好 欺负 呗 一条 一条 抡 出来 的 一个 一个 粉丝 的 努力 就 不配 呗 修改 规则 公开 声明 这么 简单 的 事情 都 做 不到 你 也 配 叫 企业 官博 第三 在 我家 质疑 规则 进行 沟通 的 时候 你们 给出 的 什么 意见 打扰 艺人 让 艺人 签名 补偿 粉丝 羊毛出在羊身上 这套 理念 二十一 世纪 过去 二十年 了 我 还 能 看到 着实 令人咋舌 我愿称 你们 为 不要脸 的 神 这波 是 在 鼓吹 什么 精神 够 无赖 就 能 无视 一切 努力 收起 全文
0.015641153
predict value: negtive

第4条，共10条
这 一年 太难 了 月 日 英国 卫生 大臣 汉考克 被 网友 发现 在 节目 中 假哭 而 遭到 批评 当天 的 节目 中当 一名 新冠 疫苗 接种 者 发表 完 感受 后 汉考克